The aim is to analysis and compare the different cities and find the one most livabl.
This is difficult to determine since its based on personal preference. we can however explore the data and she what can be found.

In [ ]:
%%capture
!pip install -U plotly

In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'


## prelim exploration of data

In [ ]:
inndf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Innsbruck_weather_2015-19.csv", parse_dates=['date'])
inndf.head()

,date,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),month,month-year,year
0,2015-01-01,-3.137500,-7.4,1.0,724.241667,92.458333,1.166667,0.0,Jan,Jan-2015,2015
1,2015-01-02,-3.341667,-9.2,1.5,721.975000,94.875000,1.125000,1.6,Jan,Jan-2015,2015
2,2015-01-03,1.458333,0.4,3.2,717.537500,96.833333,1.791667,13.1,Jan,Jan-2015,2015
3,2015-01-04,2.108333,-0.2,4.6,717.608333,79.291667,3.041667,25.9,Jan,Jan-2015,2015
4,2015-01-05,-0.841667,-5.3,2.4,721.662500,74.875000,2.666667,0.3,Jan,Jan-2015,2015


In [ ]:
inndf.dtypes

date                  datetime64[ns]
mean_temp (deg C)            float64
min_temp (deg C)             float64
max_temp (deg C)             float64
mean_pressure (mb)           float64
mean_humidity (%)            float64
mean_wind (ms-1)             float64
precip_sum (mm)              float64
month                         object
month-year                    object
year                           int64
dtype: object

In [ ]:
inndf['year'].unique()

array([2015, 2016, 2017, 2018, 2019])

In [ ]:
inndf['year'].value_counts()

2016    366
2019    365
2018    365
2017    365
2015    365
Name: year, dtype: int64

 # load  city data

In [ ]:
#inndf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Innsbruck_weather_2015-19.csv", parse_dates=['date'])
londf = pd.read_csv(r"https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/London_weather_2015-19.csv", parse_dates=['date'])
sindf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Singapore_weather_2015-19.csv', parse_dates=['date'])
syddf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Sydney_weather_2015-19.csv', parse_dates=['date'])
tehdf = pd.read_csv(r'https://raw.githubusercontent.com/ds4geo/ds4geo/master/data/timeseries/meteo/Tehran_weather_2015-19.csv', parse_dates=['date'])

#assign dfs to list for later iterating
dflist = [inndf, londf, sindf, syddf, tehdf]

#assign cities to a list for later use
citylist = ['Innsbruck', 'London', 'Singapore', 'Sydney', 'Tehran']

In [ ]:
#add the city name as a seperate column and create a single dataframe 

for df, name in zip(dflist, citylist):
  df['city'] = name
df = pd.concat(dflist)


# boxplots 

In [ ]:
#box plots made using subplots and plotly graph_objects (go)   

fig = make_subplots(rows=3, cols=2,
                    subplot_titles=("Mean Temperature (deg C)",
                                    "Mean Humidity (%)",
                                    "Mean Pressure (mb) ",
                                    "Mean Wind Speed (Meters per second)", 
                                    'Precipitation (sum) (mm)'))

fig.add_trace(
    go.Box(x=df['city'], y=df['mean_temp (deg C)']),
    row=1, col=1, 
)

fig.add_trace(
    go.Box(x=df['city'], y=df['mean_humidity (%)'] ),
    row=1, col=2
)

fig.add_trace(
    go.Box(x=df['city'], y=df['mean_pressure (mb)']),
    row=2, col=1
)

fig.add_trace(
    go.Box(x=df['city'], y=df['mean_wind (ms-1)']),
    row=2, col=2
)

fig.add_trace(
    go.Box(x=df['city'], y=df['precip_sum (mm)']),
    row=3, col=1
)

fig.update_layout(height=1000, width=1000,
                  title_text="Box plots",
                  showlegend=False
)

fig.show()

With the above box plots we can get an idea of the values of each of the variables. It can give an insight into disributions, however I think histograms can be better for distribution visualistation. There are alot of interesting trends to look into here

Interesting things of note -
  
*   the fist thing to notice is that Singapore has a very consistent daily temperature considering this is including all months. We can also see that Tehran have the most varying temperatures. 
*   We can also see that Singapore has consistant values for most of the variables, hence it appears singapore has the most consistent (stable?) weather
*   Tehran appears to be quite different in term of pressure and humidity and rainfall (although rainfall is not so readable in this plot). There is likely a relationship between these variables and I will look into this later below.
*   Innbruck is also quite different in terms of pressure, likely related to elevation, but maybe something else
*   one last comment about the precipitation: the plots look a little strange. This shows us that majority of days will have little amounts of rain, however there are days that have sonsiderably more. these are fewer and are considered outliers. This makes it difficult to visualalise. perhaps some tweaking is reqired.










In [ ]:
# one box plot made using plotly express

rainfall_adjusted = df[df['precip_sum (mm)'] >0]

fig = px.box(rainfall_adjusted, y='precip_sum (mm)', x='city', points='all')

fig.update_layout(
    title="Boxplots of daily rainfall (mm)",
    )

in the graph below I can see that London has considerable propotion of  days with rainfall in all bins below 4.4mm. Click on the legend to visualise certain cities.

In [ ]:
#using the between and quantile methods I removed the top 2%   
fig = px.histogram(rainfall_adjusted[rainfall_adjusted['precip_sum (mm)'].between(rainfall_adjusted['precip_sum (mm)'].quantile(0), rainfall_adjusted['precip_sum (mm)'].quantile(.98))], y='precip_sum (mm)', color='city', marginal="rug")

fig.update_layout(
    title="histogram of daily rainfall (mm)",
    barmode='overlay'
    )
fig.update_traces(opacity=0.75)

In [ ]:
london_rainfall = rainfall_adjusted[rainfall_adjusted['city'] =='London']

#of days with rain, get percentage of days with under 4.4mm 
len(london_rainfall[london_rainfall['precip_sum (mm)']<=4.4])/ len(london_rainfall)*100



65.46329723225031

# Temperature histograms

In [ ]:
#Histogram of mean temperature
fig = px.histogram(df, y='mean_temp (deg C)', color='city')
fig.update_layout(
    barmode='overlay',
    title="Histogram of mean temperatures (deg C)",
    xaxis_title="Number of Days",
    yaxis_title="Temperature (deg C)",
    legend_title="City",
    )

fig.update_traces(opacity=0.75)


here again we can see the consistancy of the temperature in Singapore. 
we can also visually compare the different distributions betweeen cities.

# Aproach 2. group by months

In [ ]:
#get average day by month. because we cannot keep the city collum from df, we must create seperate dfs for each city
#will set cumulative for rainfall in next step

month_ordered = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']


inndf_months = inndf.groupby(by=['month']).mean().reindex(month_ordered)
londf_months = londf.groupby(by=['month']).mean().reindex(month_ordered)
sindf_months = sindf.groupby(by=['month']).mean().reindex(month_ordered)
syddf_months = syddf.groupby(by=['month']).mean().reindex(month_ordered)
tehdf_months = tehdf.groupby(by=['month']).mean().reindex(month_ordered)

mdflist = [inndf_months, londf_months, sindf_months, syddf_months, tehdf_months]





In [ ]:
inndf_months.head()

,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),year
month,,,,,,,,
Jan,-1.036986,-4.572258,3.523226,710.995404,84.563344,2.028521,7.080000,2017.000000
Feb,1.459536,-2.696454,7.192199,710.668417,77.120303,2.071454,3.158865,2016.992908
Mar,6.061640,0.803871,12.434839,709.293662,69.261675,2.447647,4.507097,2017.000000
Apr,10.601215,4.645333,16.982667,710.002559,64.274459,2.805385,5.587333,2017.000000
May,14.001605,8.596129,20.202581,709.830970,69.726098,2.369585,10.334839,2017.000000


In [ ]:
for monthsdf, odf in zip(mdflist, dflist):
  monthsdf['precip_sum (mm)'] = odf[['month', 'precip_sum (mm)']].groupby(by=['month']).sum().reindex(month_ordered)['precip_sum (mm)']

In [ ]:
for mdf, city in zip(mdflist, citylist):
  mdf['city'] = city

In [ ]:
mdfs = pd.concat(mdflist)
mdfs.head(24)

,mean_temp (deg C),min_temp (deg C),max_temp (deg C),mean_pressure (mb),mean_humidity (%),mean_wind (ms-1),precip_sum (mm),year,city
month,,,,,,,,,
Jan,-1.036986,-4.572258,3.523226,710.995404,84.563344,2.028521,1097.4,2017.000000,Innsbruck
Feb,1.459536,-2.696454,7.192199,710.668417,77.120303,2.071454,445.4,2016.992908,Innsbruck
Mar,6.061640,0.803871,12.434839,709.293662,69.261675,2.447647,698.6,2017.000000,Innsbruck
Apr,10.601215,4.645333,16.982667,710.002559,64.274459,2.805385,838.1,2017.000000,Innsbruck
May,14.001605,8.596129,20.202581,709.830970,69.726098,2.369585,1601.9,2017.000000,Innsbruck
Jun,18.983879,13.187333,25.584000,711.072269,68.044504,2.257706,1666.0,2017.000000,Innsbruck
Jul,19.910417,14.322581,26.470323,711.496566,70.569659,1.934507,1780.3,2017.000000,Innsbruck
Aug,19.554905,13.970323,26.247097,712.556892,72.964836,2.022288,2318.1,2017.000000,Innsbruck
Sep,14.570158,9.645333,20.586000,712.863897,76.828923,1.865618,1394.3,2017.000000,Innsbruck


In [ ]:
px.line(mdfs, y='precip_sum (mm)', color='city', title='Average monthly rainfall')

In [ ]:

fig = px.scatter(df[df['city'] == 'Singapore'], y='precip_sum (mm)', x='date', color='city', trendline="ols")
fig.show()
# I can of course plot all the cities and use the interative plot to filter what I want to look at but this is not as convenient when sharing a notebook 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



From the average monthly rainfall I aslo noticed that singapore had quite a high rainfall during December. Although this could be attributed to monsoon season, the second graph shows Singapore had a significantly higher rainfall in December of 2019 as compared to other years. this therefore affects the average over all years.
Also the regresssion line interestingly shows and increase over the years.

In [ ]:
results = px.get_trendline_results(fig)
print(results)

results.query("city=='Singapore'").px_fit_results.iloc[0].summary()

        city                                     px_fit_results
0  Singapore  <statsmodels.regression.linear_model.Regressio...


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     48.85
Date:                Mon, 01 Feb 2021   Prob (F-statistic):           3.86e-12
Time:                        12:00:59   Log-Likelihood:                -7531.9
No. Observations:                1826   AIC:                         1.507e+04
Df Residuals:                    1824   BIC:                         1.508e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -74.2210     11.539     -6.432      0.000     -96.852     -51.590
x1          5.378e-08   7.69e-09      6.989      0.000    3.87e-08    6.89e-08
==============================================================================
Omnibus:                     1584.378   Durbin-Watson:                   1.279
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            45003.261
Skew:                           4.064   Prob(JB):                         0.00
Kurtosis:                      25.922   Cond. No.                     4.94e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.94e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

however the summery stats of the trend line shows its not a strong fit, r^2 = 0.026

In [ ]:
fig = px.line(mdfs, y=['mean_temp (deg C)',	'min_temp (deg C)',	'max_temp (deg C)'], color='city', title='MIN, MAX and MEAN monthly Temperatures')
fig.update_yaxes(title='Temperature (deg C)')
fig.show()



# Aproach 3. number of days per categorie e.g. days of no rain.
Idea for what to use
-no rain 
-mean temp > 15
-min temp > 0

In [ ]:
len(inndf[inndf['precip_sum (mm)'] == 0.0]) /5


165.8

In [ ]:
# Here I iterated through a list of the dataframes 'dflist'.  

NoRain = []     #no. of days without rain (annual average)
AboveZero = []  #no. of days with min temp above 0
NotWindy = []   #no. of days without srong wind

for dfl in dflist:
  NoRain.append(len(dfl[dfl['precip_sum (mm)'] == 0.0]) /5)
  AboveZero.append(len(dfl[dfl['min_temp (deg C)'] > 0]) /5)
  NotWindy.append(len(dfl[dfl['mean_wind (ms-1)'] < 4]) /5)

#this would then be added to a dataframe or just plotted


In [ ]:
#or, create a dict
stats_list = [NoRain, AboveZero, NotWindy]
stats_listSTR = ['NoRain', 'AboveZero', 'NotWindy']
citystats = {}

for name, stat in zip(stats_listSTR, stats_list):

    citystats[name] = stat


citystats
  

{'AboveZero': [263.8, 342.2, 365.2, 365.2, 350.6],
 'NoRain': [165.8, 198.0, 183.6, 205.6, 308.8],
 'NotWindy': [332.8, 183.4, 324.2, 55.6, 291.0]}

In [ ]:
#convert to DataFrame and plot    https://plotly.com/python/pandas-backend/#:~:text=Since%20version%200.25%2C%20Pandas%20has,even%20needing%20to%20import%20Plotly.

cdf = pd.DataFrame.from_dict(data= citystats, orient='columns')
cdf['city'] = citylist

cdf = cdf.set_index('city')
fig = cdf.plot.bar(facet_row='variable', title='Total days of...', labels='days')
fig.show()

note for later: barometric pressure over 1022.689 mb indicates clear and stable (google.com)
and windspeed less than 4 

In [ ]:
#exploring relationships with pressure
px.scatter(df, x='precip_sum (mm)', y='mean_pressure (mb)', color='city')

# statistical comparisons

In [ ]:
# Analysis of Variance Test (ANOVA) 
from scipy.stats import f_oneway

#compare mean temperatures between innsbruck and london
stat, p = f_oneway(inndf['precip_sum (mm)'], londf['precip_sum (mm)'])
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
	print('Not Significantly different')
else:
	print('Significantly different')


stat=159.050, p=0.000
Significantly different
